In [ ]:

print("\n--- Step 6: Monitoring Training with TensorBoard ---")
%load_ext tensorboard
%tensorboard --logdir {project_name}
print("TensorBoard launched. Check the output above for the link.")

In [ ]:

print("\n--- Step 7: Analyzing Epoch-wise Metrics ---")

results_csv_path = os.path.join('runs', 'detect', run_name, 'results.csv')

if os.path.exists(results_csv_path):
    df_results = pd.read_csv(results_csv_path)
    df_results.columns = df_results.columns.str.strip()

    print(f"Loaded metrics from: {results_csv_path}\n")

    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.plot(df_results['epoch'], df_results['train/box_loss'], label='Train Box Loss')
    plt.plot(df_results['epoch'], df_results['train/cls_loss'], label='Train Class Loss')
    plt.plot(df_results['epoch'], df_results['train/dfl_loss'], label='Train DFL Loss')
    plt.plot(df_results['epoch'], df_results['val/box_loss'], label='Val Box Loss', linestyle='--')
    plt.plot(df_results['epoch'], df_results['val/cls_loss'], label='Val Class Loss', linestyle='--')
    plt.plot(df_results['epoch'], df_results['val/dfl_loss'], label='Val DFL Loss', linestyle='--')
    plt.title('Losses per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(df_results['epoch'], df_results['metrics/mAP50(B)'], label='Val mAP50')
    plt.plot(df_results['epoch'], df_results['metrics/mAP50-95(B)'], label='Val mAP50-95')
    plt.title('Validation mAP per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 3)
    plt.plot(df_results['epoch'], df_results['metrics/precision(B)'], label='Val Precision')
    plt.plot(df_results['epoch'], df_results['metrics/recall(B)'], label='Val Recall')
    plt.title('Validation Precision & Recall per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

else:
    print(f"Error: results.csv not found at {results_csv_path}.")
    print("Please ensure the training completed successfully and check the correct run directory name.")
print("Epoch-wise metrics analysis complete.")

In [ ]:
# ================================================================
# Cell 9: Evaluate the Trained Model on the Separate Test Set
# ================================================================
print("\n--- Step 8: Evaluating Model on Test Set ---")

best_model_path = os.path.join('runs', 'detect', run_name, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    trained_model = YOLO(best_model_path)
    print(f"Loading best model from: {best_model_path}")

    metrics = trained_model.val(data=os.path.join(output_yolo_dir, "data.yaml"), split='test')

    print("\n--- Test Set Evaluation Metrics ---")
    print(f"mAP50-95 (Test): {metrics.box.map}")
    print(f"mAP50 (Test): {metrics.box.map50}")
    print(f"Precision (Test): {metrics.box.mp}")
    print(f"Recall (Test): {metrics.box.mr}")
    print("-----------------------------------")
else:
    print(f"Error: Best model not found at {best_model_path}. Cannot evaluate on test set.")
print("Test set evaluation complete.")

In [ ]:
# ================================================================
# Cell 10: Visualize Predicted Bounding Boxes on Test Images
# ================================================================
print("\n--- Step 9: Visualizing Predicted Bounding Boxes ---")

test_images_dir = os.path.join(output_yolo_dir, 'images', 'test')

if not os.path.exists(test_images_dir):
    print(f"Error: Test images directory not found at {test_images_dir}.")
    print("Cannot perform visualization without test images.")
else:
    test_image_files = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if not test_image_files:
        print(f"No image files found in {test_images_dir}. Cannot perform visualization.")
    else:
        num_images_to_visualize = 5
        selected_images = random.sample(test_image_files, min(num_images_to_visualize, len(test_image_files)))

        print(f"Visualizing detections on {len(selected_images)} random test images...")

        for img_filename in selected_images:
            img_path = os.path.join(test_images_dir, img_filename)

            print(f"\nProcessing image: {img_filename}")

            results = trained_model(img_path)

            for r in results:
                im_bgr = r.plot()
                im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)

                plt.figure(figsize=(15, 10))
                plt.imshow(im_rgb)
                plt.axis('off')
                plt.title(f'Detected Car Plates on {img_filename}')
                plt.show()

                boxes = r.boxes.xyxy.cpu().numpy()
                scores = r.boxes.conf.cpu().numpy()
                class_ids = r.boxes.cls.cpu().numpy()

                if len(boxes) > 0:
                    print(f"  Detections for {img_filename}:")
                    for i, box in enumerate(boxes):
                        x1, y1, x2, y2 = map(int, box)
                        score = scores[i]
                        class_name = trained_model.names[int(class_ids[i])]
                        print(f"    - {class_name}, Confidence: {score:.2f}, Box: [{x1}, {y1}, {x2}, {y2}]")
                else:
                    print(f"  No car plates detected in {img_filename}.")
print("Visualization complete.")

print("\nScript finished.")